In [31]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [32]:
df_movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv') 
df_credits = pd.read_csv('../../datasets/tmdb_5000_credits.csv') 
df_credits.rename(columns={'movie_id': 'id'}, inplace=True)
movies = df_movies.merge(df_credits,on='id')
movies = movies[(movies['status'] == 'Released') & (movies['overview'].isna()==False) & (movies['genres'].isna()==False) & (movies['keywords'].isna()==False)]
#movies=movies[movies['overview'].isna()==False]
#print(movies.isnull().sum())
print('Количество фильмов = ',movies.shape[0])

#новый столбец для фильтрации
movies['year'] = movies['release_date'].astype(str).str[:-6].fillna('')
movies['year']=movies['year'].replace('', 'нет данных о годе')
#print(movies['year'][movies['id']==380097])
movies

Количество фильмов =  4792


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew,year
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",...,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",1992
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],...,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Newlyweds,"[{""cast_id"": 

## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [34]:
movies['keywords'].iloc[0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [35]:
#объедим выбранные текстовые поля описания фильма (overview) и ключевых слов к фильму (keywords)
movies['overview'].fillna('')

#в keywords у нас список словарей, выташим только теги, для этого напишем функцию 
def extract_tags(s):
    list_of_dict = json.loads(s)
    tags=[d["name"] for item in list_of_dict for d in (item if isinstance(item, list) else [item])]
    result = ", ".join(tags)
    return result

movies['keywords2'] = movies['keywords'].apply(extract_tags)
movies['full_description'] = movies[['overview', 'keywords2']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
movies['full_description'].iloc[0]


'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d'

In [36]:
vectorizer = TfidfVectorizer(stop_words = 'english', max_features=10000)
tfidf_matrix = vectorizer.fit_transform(movies['full_description'], )
feature_names = vectorizer.get_feature_names_out()
print('размерность матрицы', tfidf_matrix.shape)

размерность матрицы (4792, 10000)


## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [37]:
cosine_sim = linear_kernel(tfidf_matrix)
cosine_sim.shape
cos_sim_df = pd.DataFrame(cosine_sim, columns=movies['id'], index=movies['id'])
cos_sim_df

id,19995,285,206647,49026,49529,559,38757,99861,767,209112,...,182291,286939,124606,14337,67238,9367,72766,231617,126186,25975
id,,,,,,,,,,,,,,,,,,,,,
19995,1.000000,0.006698,0.000000,0.011519,0.193628,0.043216,0.008125,0.048032,0.000000,0.012418,...,0.000000,0.000000,0.017703,0.033747,0.000000,0.000000,0.0,0.004940,0.013178,0.000000
285,0.006698,1.000000,0.000000,0.014123,0.040554,0.023146,0.004609,0.014998,0.000000,0.000000,...,0.000000,0.000000,0.014527,0.000000,0.000000,0.009755,0.0,0.019802,0.000000,0.000000
206647,0.000000,0.000000,1.000000,0.018515,0.014561,0.014341,0.019677,0.047230,0.016920,0.006142,...,0.031525,0.000000,0.000000,0.000000,0.017365,0.014662,0.0,0.011729,0.000000,0.000000
49026,0.011519,0.014123,0.018515,1.000000,0.006272,0.024840,0.024703,0.040374,0.011192,0.226748,...,0.006161,0.000000,0.004955,0.018283,0.010281,0.000000,0.0,0.028046,0.035781,0.017660
49529,0.193628,0.040554,0.014561,0.006272,1.000000,0.020283,0.020632,0.056310,0.000000,0.014569,...,0.005251,0.000000,0.000000,0.010926,0.000000,0.000000,0.0,0.004216,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9367,0.000000,0.009755,0.014662,0.000000,0.000000,0.000000,0.000000,0.010251,0.020069,0.000000,...,0.000000,0.013357,0.000000,0.000000,0.003402,1.000000,0.0,0.000000,0.008597,0.000000
72766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
231617,0.004940,0.019802,0.011729,0.028046,0.004216,0.024502,0.000000,0.017785,0.022825,0.012889,...,0.005563,0.000000,0.015358,0.029665,0.000000,0.000000,0.0,1.000000,0.017167,0.024581


In [38]:
#способ2
from sklearn.metrics.pairwise import cosine_similarity

# Рассчитываем матрицу cosine similarity
cos_sim_matrix = cosine_similarity(tfidf_matrix)

# Создаем DataFrame с названиями колонок и индексов, соответствующими id фильмов
cos_sim_df2 = pd.DataFrame(cos_sim_matrix, columns=movies['id'], index=movies['id'])


In [39]:
(movies['original_title']==movies['title_x']).value_counts()

True     4531
False     261
Name: count, dtype: int64

In [40]:
(movies['title_y']==movies['title_x']).value_counts()
#следовательно, один столбец можно удалить

True    4792
Name: count, dtype: int64

In [41]:
#переназовем столбец title_x и удалим title_y
movies.rename(columns={'title_x': 'title'}, inplace=True)
movies.drop(columns = ['title_y'],axis = 1, inplace=True)
cos_sim_df = cos_sim_df.rename_axis('movie_id')
print(cos_sim_df.columns[0])


19995


In [43]:
cos_sim_df.to_csv (r'../assets/distance.csv', index= True)
movies.to_csv (r'../assets/movies.csv', index= False)

# Далее идет проверка функций,черновики

In [13]:
#проверяем, как работает
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):

    indices = pd.Series(movies_dataset.index, index=movies_dataset['title']) #connecting the rows in the sparse matrix with the titles (same rows)
    idx= indices[title]
# Получаем оценки сходство попарно
    sim_scores = list(enumerate(cosine_sim[idx]))
# Сортируем фильмы по показателям сходство
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
# Получите оценки за top_k наиболее похожих фильмов
    sim_scores = sim_scores[1:top_k + 1]
# Получить индексы фильмов
    movie_indices = [i[0] for i in sim_scores]
# Возвращает список лучших похожих фильмов
    return movies_dataset['title'].iloc[movie_indices]

In [21]:
#Проверка работы
aaa='''The Pirates! In an Adventure with Scientists!'''
get_recommendations(movies_dataset=movies, title=aaa, cosine_sim=cosine_sim,top_k = 5)

3820                                           The Pirate
199     Pirates of the Caribbean: The Curse of the Bla...
2590       VeggieTales: The Pirates Who Don't Do Anything
1709                         Space Pirate Captain Harlock
340                                      Cutthroat Island
Name: title, dtype: object

In [28]:
#проверка фильтрации
moviesfiltred=movies.copy()
moviesfiltred=moviesfiltred[moviesfiltred['genres'].astype(str).str.contains('Family')]
moviesfiltred=moviesfiltred[moviesfiltred['year'].astype(int)>=2015]
print(len(moviesfiltred))
moviesfiltred



27


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,status,tagline,title,vote_average,vote_count,cast,crew,year,keywords2,full_description
54,175000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 16, ""...",http://movies.disney.com/the-good-dinosaur,105864,"[{""id"": 1720, ""name"": ""tyrannosaurus rex""}, {""...",en,The Good Dinosaur,An epic journey into the world of dinosaurs wh...,51.692953,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,Released,Little Arms With Big Attitude,The Good Dinosaur,6.6,1736,"[{""cast_id"": 20, ""character"": ""Arlo (voice)"", ...","[{""credit_id"": ""52fe4a58c3a36847f81c8a07"", ""de...",2015,"tyrannosaurus rex, friends, alternate history,...",An epic journey into the world of dinosaurs wh...
77,175000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",http://movies.disney.com/inside-out,150540,"[{""id"": 1566, ""name"": ""dream""}, {""id"": 6513, ""...",en,Inside Out,"Growing up can be a bumpy road, and it's no ex...",128.655964,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,Released,Meet the little voices inside your head.,Inside Out,8.0,6560,"[{""cast_id"": 4, ""character"": ""Joy (voice)"", ""c...","[{""credit_id"": ""56098866c3a368681000003a"", ""de...",2015,"dream, cartoon, imaginary friend, animation, f...","Growing up can be a bumpy road, and it's no ex..."
78,175000000,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 12, ""...",http://movies.disney.com/the-jungle-book-2016,278927,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,The Jungle Book,After a threat from the tiger Shere Khan force...,94.199316,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,Released,NaN,The Jungle Book,6.7,2892,"[{""cast_id"": 12, ""character"": ""Mowgli"", ""credi...","[{""credit_id"": ""571507b692514105a80032f0"", ""de...",2016,"based on novel, snake, wolf, elephant, tiger, ...",After a threat from the tiger Shere Khan force...
87,190000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",http://movies.disney.com/tomorrowland,158852,"[{""id"": 1436, ""name"": ""inventor""}, {""id"": 1233...",en,Tomorrowland,"Bound by a shared destiny, a bright, optimisti...",130.311355,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,Released,Imagine a world where nothing is impossible.,Tomorrowland,6.2,2846,"[{""cast_id"": 17, ""character"": ""Casey Newton"", ...","[{""credit_id"": ""55bb85e6c3a36869b400723c"", ""de...",2015,"inventor, apocalypse, destiny, imax, dreamer, ...","Bound by a shared destiny, a bright, optimisti..."
143,150000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",NaN,266647,"[{""id"": 334, ""name"": ""flying""}, {""id"": 2343, ""...",en,Pan,Living a bleak existence at a London orphanage...,48.035280,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...",...,Released,Every legend has a beginning.,Pan,5.9,954,"[{""cast_id"": 9, ""character"": ""Peter Pan"", ""cre...","[{""credit_id"": ""57373a28925141136e003ec3"", ""de...",2015,"flying, magic, fairy tale, peter pan, mermaid,...",Living a bleak existence at a London orphanage...
152,145000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.kungfupanda.com/,140300,"[{""id"": 478, ""name"": ""china""}, {""id"": 779, ""na...",en,Kung Fu Panda 3,"Continuing his ""legendary adventures of awesom...",56.747978,"[{""name"": ""Twentieth Century Fox Film Corporat...",...,Released,Grab destiny by the rice dumplings.,Kung Fu Panda 3,6.7,1603,"[{""cast_id"": 4, ""character"": ""Po (voice)"", ""cr...","[{""credit_id"": ""596107c49251415a4008f021"", ""de...",2016,"china, martial arts, kung fu, village, panda, ...","Continuing his ""legendary adventures of awesom..."
175,140000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...",http://movies.disney.com/the-bfg,267935,"[{""id"": 212, ""name"": ""london england""}, {""id"":...",en,The BFG,The BFG is no ord

In [44]:
sorted=moviesfiltred['id'].tolist()

In [30]:
idx = int(movies[movies['title'] == '''Pirates of the Caribbean: At World's End'''].iloc[0]['id'])

print(idx)

distancefiltred=cos_sim_df.copy()
distancefiltred.index = distancefiltred.index.astype(int)
distancefiltred.columns = distancefiltred.columns.astype(int)
if idx in sorted:
            sorted.remove(idx)
print(type(sorted[0]))
sorted.append(idx)
print(len(distancefiltred.index))
print(sorted)
distancefiltred = distancefiltred.loc[sorted, sorted]
distancefiltred

285
<class 'int'>
4792
[105864, 150540, 278927, 158852, 266647, 140300, 267935, 228161, 150689, 258509, 159824, 328111, 211672, 153518, 228165, 256961, 272878, 302688, 333348, 294272, 447027, 426469, 361475, 357441, 379532, 371690, 306667, 285]


id,105864,150540,278927,158852,266647,140300,267935,228161,150689,258509,...,333348,294272,447027,426469,361475,357441,379532,371690,306667,285
movie_id,,,,,,,,,,,,,,,,,,,,,
105864,1.000000,0.023872,0.053974,0.000000,0.025019,0.031265,0.000000,0.014019,0.000000,0.000000,...,0.031238,0.016260,0.000000,0.000000,0.000000,0.031595,0.000000,0.042155,0.000000,0.000000
150540,0.023872,1.000000,0.005051,0.005406,0.011310,0.000000,0.004117,0.013444,0.004429,0.039793,...,0.017231,0.036274,0.033656,0.020403,0.018583,0.021914,0.000000,0.006855,0.019272,0.018860
278927,0.053974,0.005051,1.000000,0.000000,0.044258,0.082333,0.037491,0.009358,0.023145,0.100354,...,0.000000,0.097571,0.011376,0.000000,0.000000,0.000000,0.000000,0.019059,0.000000,0.000000
158852,0.000000,0.005406,0.000000,1.000000,0.056080,0.000000,0.000000,0.041856,0.000000,0.013841,...,0.000000,0.000000,0.000000,0.032336,0.031900,0.000000,0.030164,0.019086,0.014599,0.000000
266647,0.025019,0.011310,0.044258,0.056080,1.000000,0.000000,0.009194,0.008956,0.064962,0.012012,...,0.000000,0.026219,0.019201,0.085198,0.014297,0.000000,0.000000,0.053306,0.017181,0.051430
140300,0.031265,0.000000,0.082333,0.000000,0.000000,1.000000,0.024409,0.008537,0.000000,0.060924,...,0.020045,0.086530,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023058,0.008271
267935,0.000000,0.004117,0.037491,0.000000,0.009194,0.024409,1.000000,0.006194,0.028184,0.035000,...,0.011277,0.059949,0.022316,0.010686,0.000000,0.000000,0.040298,0.000000,0.027530,0.000000
228161,0.014019,0.013444,0.009358,0.041856,0.008956,0.008537,0.006194,1.000000,0.007181,0.011505,...,0.000000,0.022346,0.022790,0.010531,0.000000,0.060898,0.000000,0.015307,0.056743,0.012580
150689,0.000000,0.004429,0.023145,0.000000,0.064962,0.000000,0.028184,0.007181,1.000000,0.000000,...,0.000000,0.050259,0.009975,0.020277,0.007785,0.000000,0.000000,0.012535,0.007261,0.000000


In [ ]:
print(moviesfiltred['year'][moviesfiltred['id']==380097])
print(len(moviesfiltred))
moviesfiltred.drop(moviesfiltred[moviesfiltred['year'] == 'нет данных о годе'].index, inplace=True)
print(len(moviesfiltred))
moviesfiltred=moviesfiltred[moviesfiltred['year'].astype(int)>=2016]
sorted=moviesfiltred['id'].tolist()
#sorted=moviesfiltred.index.tolist()
print(len(sorted))


In [116]:
def sublist_in_list(sub, lis):
    return str(sub).strip('[]') in str(lis).strip('[]')
    
sublist_in_list(sorted, distancefiltred.index) # Входит ли list_1 в list_2 ?

False